# Investigate

## Graph Investigattion

In [1]:
import os
import onnx

model_name = "qwen25_vli_visual.cpu.float32.onnx-dynamo.onnx"
print(os.path.abspath(model_name))
model_onnx = onnx.load(model_name)

model_name_custom = "qwen25_vli_visual.cpu.float32.custom.onnx"
print(os.path.abspath(model_name_custom))
model_onnx_custom = onnx.load(model_name_custom)

/home/xadupre/github/onnx-diagnostic/_scripts/qwen25_vli_visual.cpu.float32.onnx-dynamo.onnx
/home/xadupre/github/onnx-diagnostic/_scripts/qwen25_vli_visual.cpu.float32.custom.onnx


In [2]:
from onnx_diagnostic.helpers.onnx_helper import pretty_onnx

for node in model_onnx.graph.node:
    if node.name == "n16":
        print(pretty_onnx(node))
        g = node.attribute[0].g
        break

for node in model_onnx_custom.graph.node:
    if node.name == "n16":
        print(pretty_onnx(node))
        g_custom = node.attribute[0].g
        break

Loop(num_patches, , seq_attn) -> seq_attn_6
Loop(num_patches, , seq_attn) -> seq_attn_6


In [3]:
g.input

[name: "i_patch"
type {
  tensor_type {
    elem_type: 7
    shape {
    }
  }
}
, name: "cond_in"
type {
  tensor_type {
    elem_type: 9
    shape {
    }
  }
}
, name: "seq_attn_3"
type {
  tensor_type {
    elem_type: 1
  }
}
]

In [4]:
g.output

[name: "cond_out"
type {
  tensor_type {
    elem_type: 9
    shape {
    }
  }
}
, name: "seq_attn_5"
]

In [5]:
g_custom.input

[name: "i_patch"
type {
  tensor_type {
    elem_type: 7
    shape {
    }
  }
}
, name: "cond_in"
type {
  tensor_type {
    elem_type: 9
    shape {
    }
  }
}
, name: "seq_attn_3"
type {
}
]

In [6]:
g_custom.output

[name: "cond_out"
type {
  tensor_type {
    elem_type: 9
    shape {
    }
  }
}
, name: "seq_attn_5"
type {
}
]

In [12]:
def remove_inplace_body_last_input_output_type_for_loop(filename: str):
    model = onnx.load(filename, load_external_data=False)
    for node in model.graph.node:
        if node.op_type == "Loop":
            g = node.attribute[0].g
            g.input[-1].type.CopyFrom(onnx.TypeProto())
            g.output[-1].type.CopyFrom(onnx.TypeProto())
    onnx.save(model, filename, save_as_external_data=False)


remove_inplace_body_last_input_output_type_for_loop(model_name)

## Statistics

In [18]:
import pandas

In [19]:
index = [
    "model_id",
    "pretrained",
    "device",
    "dtype",
    "attention",
    "opset",
    "exporter",
]
values = ["abs", "%>0.1", "%>0.01", "export_duration", "speedup", "latency_ort_n"]
df = pandas.read_json("dump_models/collection_statistics.js", lines=True)
stat = df[[*index, *values]].groupby(index).max()
stat

KeyError: "['latency_ort_n'] not in index"

In [17]:
df = pandas.read_json("dump_models/collection_statistics.js", lines=True)
stat = df[df.exporter == "onnx-dynamo"][[*index, *values]].groupby(index).max()
stat

abs  \
model_id                    pretrained device dtype   attention opset exporter                 
Qwen/Qwen2.5-VL-7B-Instruct True       cpu    float16 LOOPA23   23    onnx-dynamo   4.835938   
                                                      LOOPMHA   22    onnx-dynamo   3.640625   
                                       cuda   float16 LOOPMHA   22    onnx-dynamo  57.328125   
                                                      PACKED    22    onnx-dynamo   6.578125   
microsoft/Fara-7B           True       cpu    float16 LOOPA23   23    onnx-dynamo  10.031250   
                                                      LOOPMHA   22    onnx-dynamo   3.812500   
                                       cuda   float16 LOOPMHA   22    onnx-dynamo  41.994141   
                                                      PACKED    22    onnx-dynamo   3.007812   

                                                                                      %>0.1  \
model_id                    pretrained device dtype   attention opset exporter                
Qwen/Qwen2.5-VL-7B-Instruct True       cpu    float16 LOOPA23   23    onnx-dynamo  0.011979   
                                                      LOOPMHA   22    onnx-dynamo  0.000931   
                                       cuda   float16 LOOPMHA   22    onnx-dynamo  0.901577   
                                                      PACKED    22    onnx-dynamo  0.001812   
microsoft/Fara-7B           True       cpu    float16 LOOPA23   23    onnx-dynamo  0.057682   
                                                      LOOPMHA   22    onnx-dynamo  0.000656   
                                       cuda   float16 LOOPMHA   22    onnx-dynamo  0.836432   
                                                      PACKED    22    onnx-dynamo  0.000508   

                                                                                     %>0.01  \
model_id                    pretrained device dtype   attention opset exporter                
Qwen/Qwen2.5-VL-7B-Instruct True       cpu    float16 LOOPA23   23    onnx-dynamo  0.487184   
                                                      LOOPMHA   22    onnx-dynamo  0.070157   
                                       cuda   float16 LOOPMHA   22    onnx-dynamo  0.990082   
                                                      PACKED    22    onnx-dynamo  0.094612   
microsoft/Fara-7B           True       cpu    float16 LOOPA23   23    onnx-dynamo  0.721280   
                                                      LOOPMHA   22    onnx-dynamo  0.029810   
                                       cuda   float16 LOOPMHA   22    onnx-dynamo  0.983630   
                                                      PACKED    22    onnx-dynamo  0.034595   

                                                                                   export_duration  \
model_id                    pretrained device dtype   attention opset exporter                       
Qwen/Qwen2.5-VL-7B-Instruct True       cpu    float16 LOOPA23   23    onnx-dynamo        37.809274   
                                                      LOOPMHA   22    onnx-dynamo        42.182219   
                                       cuda   float16 LOOPMHA   22    onnx-dynamo        42.195153   
                                                      PACKED    22    onnx-dynamo        47.045363   
microsoft/Fara-7B           True       cpu    float16 LOOPA23   23    onnx-dynamo        37.988508   
                                                      LOOPMHA   22    onnx-dynamo        41.783146   
                                       cuda   float16 LOOPMHA   22    onnx-dynamo        41.479114   
                                                      PACKED    22    onnx-dynamo        47.271413   

                                                                                    speedup  
model_id                    pretrained device dtype   attention opset exporter               
Qwen/Qwen2.5-VL-7